## Topic: Sustainable risk preferences on asset allocation: a higher order optimal portfolio study
# THI DIEU HUONG NGUYEN

#### - Data is directly imported into Jupyternotebook by using Yahoo finance API, hence the code and data are in 1 single ipynb file. Use the option ‘Restart kernel and run all cell’ is suggested, in case ‘run all cell’ option does not work.
#### - Paper and its supplementary file are submitted together with the code file.
#### - Regarding methodology implemented in code. E.g: GARCH (1,1) and GO-GARCH, please read the methodology given in the paper and its supplementary file given by the authors.
#### - All results in the presentation are taken from the finance_project_official.ipynb. Data in the tables are collected in the results cell manually,e.g: collecting results of GARCH (1,1), since copying the whole GARCH model results and paste into the presentation slides is not a good idea
#### - Paper source: https://doi.org/10.1016/j.jbef.2024.100887

In [ ]:
import yfinance as yf
import pandas as pd

import numpy as np
!pip install matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import kurtosis as scipy_kurtosis, skew as scipy_skew, jarque_bera
import statsmodels.api as sm
from scipy.stats import norm, gaussian_kde, invgauss
import seaborn as sns
import pylab
!pip install arch
from arch import arch_model
from statsmodels.tsa.ar_model import AutoReg
from scipy.optimize import minimize
!pip install xlrd
!pip install pandas openpyxl
from sklearn.decomposition import FastICA
from statsmodels.regression.rolling import RollingOLS
from scipy.linalg import eigh

In [ ]:
import yfinance as yf
print(yf.__version__)

In [ ]:
!pip install tensorflow
import tensorflow as tf

# load data and calculate log return

In [ ]:
# IESE.AS belong to euro area is not included 
sri_ticker = [ "PBW","CXSE","FAN"] 
sri_data = yf.download(sri_ticker,start="2013-01-01", end="2017-12-31")
sri_datatest = yf.download(sri_ticker,start="2018-01-01", end="2019-12-31")
sri_data_full = yf.download(sri_ticker,start="2013-01-01", end="2019-12-31")

In [ ]:
for sri in sri_data:
    sri_log_returns = np.log(sri_data['Close'] / sri_data['Close'].shift(1))
for sri in sri_datatest:
    sri_log_returnstest = np.log(sri_datatest['Close'] / sri_datatest['Close'].shift(1))
for sri in sri_data_full:
    sri_data_full_returns = np.log(sri_data_full['Close'] / sri_data_full['Close'].shift(1))

In [ ]:
# IMAE belong to euro area is not included 
traditional_ticker = ['URTH', 'SPY','EWH'] 
traditional_data = yf.download(traditional_ticker,start="2013-01-01", end="2017-12-31")
traditional_datatest = yf.download(traditional_ticker,start="2018-01-01", end="2019-12-31")
traditional_data_full = yf.download(traditional_ticker,start="2013-01-01", end="2019-12-31")

In [ ]:
for tra in traditional_data:
    tra_log_returns = np.log(traditional_data['Close'] / traditional_data['Close'].shift(1))
    
for tra in traditional_datatest:
    tra_log_returnstest = np.log(traditional_datatest['Close'] / traditional_datatest['Close'].shift(1))
    
for tra in traditional_data_full:
    traditional_data_full_returns = np.log(traditional_data_full['Close'] / traditional_data_full['Close'].shift(1))

# descriptive statistic of log returns

### SUSTAINABLE(SRI)

In [ ]:
sri_log_returns=pd.DataFrame(sri_log_returns).dropna()
sri_log_returns.index = pd.date_range(start=sri_log_returns.index[0], periods=len(sri_log_returns), freq='B')


sri_log_returnstest=pd.DataFrame(sri_log_returnstest).dropna()
sri_log_returnstest.index = pd.date_range(start=sri_log_returnstest.index[0], periods=len(sri_log_returnstest), freq='B')

sri_data_full_returns=pd.DataFrame(sri_data_full_returns).dropna()
sri_data_full_returns.index = pd.date_range(start=sri_data_full_returns.index[0], periods=len(sri_data_full_returns), freq='B')

In [ ]:
print(len(sri_log_returns))
print(len(sri_data_full_returns))

In [ ]:
# statistics of sri in sample returns
print(sri_log_returns.describe())

In [ ]:
# Skewness and Kurtosis
skewness_sri = sri_log_returns.skew()
kurtosis_sri = sri_log_returns.kurtosis()
print("Skewness:\n", skewness_sri)
print("Kurtosis:\n", kurtosis_sri)

In [ ]:
correlation_matrix_sri = sri_log_returns.corr()
print("Correlation Matrix:\n", correlation_matrix_sri)

In [ ]:
# sri_ticker = [ 'PBW','CXSE','FAN'] 
plt.figure(figsize=(10, 5))
# First subplot for 'URTH'
plt.subplot(3, 1, 1)  # 3 rows, 1 column, 1st subplot
plt.plot(sri_log_returns['CXSE'], label='CXSE ')
plt.title('CXSE Log Returns')
plt.legend()  # Ensure the legend is called right after plotting
# Second subplot for 'SPY'
plt.subplot(3, 1, 2)  # 3 rows, 1 column, 2nd subplot
plt.plot(sri_log_returns['FAN'], label='FAN ', color='red')
plt.title('FAN Log Returns')
plt.legend()  # Ensure the legend is called right after plotting
# Third subplot for 'EWH'
plt.subplot(3, 1, 3)  # 3 rows, 1 column, 3rd subplot
plt.plot(sri_log_returns['PBW'], label='PBW', color='green')
plt.title('PBW Log Returns')
plt.legend()  # Ensure the legend is called right after plotting
# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
colors = ['blue', 'red', 'green']  # Adjusted color order to your specification

plt.figure(figsize=(6, 8))  # Adjust the figure size to accommodate multiple subplots

# Loop through each column and create a separate subplot for each
for i, column in enumerate(sri_log_returns.columns):
    plt.subplot(len(sri_log_returns.columns), 1, i + 1)  # Create a subplot for each asset
    plt.hist(sri_log_returns[column], bins=20, alpha=0.75, color=colors[i], label=column)
    plt.title(f'Histogram of Returns for {column}')
    plt.xlabel('Returns')
    plt.ylabel('Frequency')
    plt.legend()

plt.tight_layout()  # Adjust layout to prevent overlap of subplot elements
plt.show()

### TRADITIONAL(TRA)

In [ ]:
tra_log_returns=pd.DataFrame(tra_log_returns).dropna()
tra_log_returns.index = pd.date_range(start=tra_log_returns.index[0], periods=len(tra_log_returns), freq='B')

tra_log_returnstest=pd.DataFrame(tra_log_returnstest).dropna()
tra_log_returnstest.index = pd.date_range(start=tra_log_returnstest.index[0], periods=len(tra_log_returnstest), freq='B')

traditional_data_full_returns=pd.DataFrame(traditional_data_full_returns).dropna()
traditional_data_full_returns.index = pd.date_range(start=traditional_data_full_returns.index[0], periods=len(traditional_data_full_returns), freq='B')

In [ ]:
print(len(tra_log_returns))
print(len(traditional_data_full_returns))

In [ ]:
# statistics of tra in sample returns
print(tra_log_returns.describe())

In [ ]:
# Skewness and Kurtosis
skewness = tra_log_returns.skew()
kurtosis = tra_log_returns.kurtosis()
print("Skewness:\n", skewness)
print("Kurtosis:\n", kurtosis)

In [ ]:
correlation_matrix_tra = tra_log_returns.corr()
print("Correlation Matrix:\n", correlation_matrix_tra)

In [ ]:
# traditional_ticker = ['EWH', 'SPY','URTH'] 
plt.figure(figsize=(10, 5))
# First subplot for 'URTH'
plt.subplot(3, 1, 1)  # 3 rows, 1 column, 1st subplot
plt.plot(tra_log_returns['EWH'], label='EWH')
plt.title('URTH Log Returns')
plt.legend()  # Ensure the legend is called right after plotting
# Second subplot for 'SPY'
plt.subplot(3, 1, 2)  # 3 rows, 1 column, 2nd subplot
plt.plot(tra_log_returns['SPY'], label='SPY', color='red')
plt.title('SPY Log Returns')
plt.legend()  # Ensure the legend is called right after plotting
# Third subplot for 'EWH'
plt.subplot(3, 1, 3)  # 3 rows, 1 column, 3rd subplot
plt.plot(tra_log_returns['URTH'], label='URTH', color='green')
plt.title('EWH Log Returns')
plt.legend()  # Ensure the legend is called right after plotting
# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
colors = ['blue', 'red', 'green']  # Adjusted color order to your specification

plt.figure(figsize=(6, 8
                   ))  # Adjust the figure size to accommodate multiple subplots

# Loop through each column and create a separate subplot for each
for i, column in enumerate(tra_log_returns.columns):
    plt.subplot(len(tra_log_returns.columns), 1, i + 1)  # Create a subplot for each asset
    plt.hist(tra_log_returns[column], bins=20, alpha=0.75, color=colors[i], label=column)
    plt.title(f'Histogram of Returns for {column}')
    plt.xlabel('Returns')
    plt.ylabel('Frequency')
    plt.legend()

plt.tight_layout()  # Adjust layout to prevent overlap of subplot elements
plt.show()

# GARCh (1,1) for each ETF in 2 Portfolio SRI & TRADITIONAL
# Conditional variances

In [ ]:
#univariate GARCH(1,1)
def fit_garch_model(log_returns, etf_name):
    # Initialize the GARCH 1,1 model
    model = arch_model(log_returns, mean='Zero', vol='GARCH', p=1, q=1, rescale=False)
    # Fit the model
    model_fit = model.fit(disp='off')
    # Print the summary of the model
    print(f"GARCH(1,1) Model Summary for {etf_name}:")
    print( model_fit.summary())
    #print(f'conditional variance for {etf_name}:')
    conditional_variances= model_fit.conditional_volatility**2
    print('======')
    return conditional_variances

In [ ]:
# sri_ticker = [ 'PBW','CXSE','FAN']
# GARCH (1,1) in sample
conditional_variances_pbw=fit_garch_model(sri_log_returns['PBW'], 'PBW')
conditional_variances_cxse=fit_garch_model(sri_log_returns['CXSE'], 'CXSE')
conditional_variances_fan=fit_garch_model(sri_log_returns['FAN'], 'FAN')

In [ ]:
# sri_ticker = [ 'PBW','CXSE','FAN']
# GARCH (1,1) out-of sample
print('out of sample sri')
conditional_variances_pbw_test=fit_garch_model(sri_log_returnstest['PBW'], 'PBW')
conditional_variances_cxse_test=fit_garch_model(sri_log_returnstest['CXSE'], 'CXSE')
conditional_variances_fan_test=fit_garch_model(sri_log_returnstest['FAN'], 'FAN')

In [ ]:
# GARCH (1,1) in sample
# traditional_ticker = ['URTH', 'SPY','EWH']
conditional_variances_urth=fit_garch_model(tra_log_returns['URTH'], 'URTH')
conditional_variances_spy=fit_garch_model(tra_log_returns['SPY'], 'SPY')
conditional_variances_ewh=fit_garch_model(tra_log_returns['EWH'], 'EWH')

In [ ]:
# GARCH (1,1) out of sample
# traditional_ticker = ['URTH', 'SPY','EWH']
conditional_variances_urth_test=fit_garch_model(tra_log_returnstest['URTH'], 'URTH')
conditional_variances_spy_test=fit_garch_model(tra_log_returnstest['SPY'], 'SPY')
conditional_variances_ewh_test=fit_garch_model(tra_log_returnstest['EWH'], 'EWH')

# estimate lamda for each ETF in the sample period

In [ ]:
def ols_regression(data, dependent_var, conditional_variances):
    # Create a temporary copy of data for manipulations to avoid altering the original DataFrame
    temp_data = data.copy()
    # Calculate 1-lagged return for the independent variable and store in the temporary DataFrame
    temp_data['lagged_return'] = temp_data[dependent_var].shift(1)
    # Attach the pre-computed conditional variances to the temporary DataFrame
    temp_data['conditional_variances'] = conditional_variances
    # Ensure there are no NaN values by dropping rows with NaNs in the newly created columns and the dependent variable
    temp_data.dropna(subset=['lagged_return', 'conditional_variances', dependent_var], inplace=True)
    
    # Prepare independent variables (including constant for intercept)
    X = sm.add_constant(temp_data[['lagged_return', 'conditional_variances']])
    
    # Define the dependent variable using the cleaned temporary DataFrame
    y = temp_data[dependent_var]
    
    # Fit the OLS regression model
    model_fit = sm.OLS(y, X).fit()
    lambda_estimate = model_fit.params.get('conditional_variances', None)
    print(f"Estimated λ-coefficient for {dependent_var}: {lambda_estimate}")
    print(model_fit.summary())
    print('***')
    return model_fit
    return lambda_estimate

In [ ]:
# sri_ticker = [ 'PBW','CXSE','FAN']
# regression of in sample
rraPBW = ols_regression(
    data=sri_log_returns,
    dependent_var='PBW',
    conditional_variances=conditional_variances_pbw)
rraCXSE = ols_regression(
    data=sri_log_returns,
    dependent_var='CXSE',
    conditional_variances=conditional_variances_cxse)
rraFAN = ols_regression(
    data=sri_log_returns,
    dependent_var='FAN',
    conditional_variances=conditional_variances_fan)

In [ ]:
# traditional_ticker = ['URTH', 'SPY','EWH']
# regression in sample
rraURTH = ols_regression(
    data=tra_log_returns,
    dependent_var='URTH',
    conditional_variances=conditional_variances_urth)
rraSPY = ols_regression(
    data=tra_log_returns,
    dependent_var='SPY',
    conditional_variances=conditional_variances_spy)
rraEWH = ols_regression(
    data=tra_log_returns,
    dependent_var='EWH',
    conditional_variances=conditional_variances_ewh)

# forecast lamda of FAN & PSY in out of sample period by choose the fit model, lamda of FAN & SPY to estimate 

In [ ]:
def forecast_lambda(data, dependent_var, conditional_variances, model_fit):
    # Prepare the new predictor data
    new_data = data.copy()
    new_data['lagged_return'] = new_data[dependent_var].shift(1)  # Assuming 'data' is a DataFrame with log returns as its main column
    new_data['conditional_variances'] = pd.Series(conditional_variances, index=new_data.index)
    # Drop rows with NaN values which might occur due to shifting
    new_data.dropna(inplace=True)
    # Prepare the X matrix for prediction
    X_new = sm.add_constant(new_data[['lagged_return', 'conditional_variances']])
    # Predict new lambda values using the fitted model
    new_data['predicted_lambda'] = model_fit.predict(X_new)
    return new_data[['predicted_lambda']]

In [ ]:

# sri_log_returnstest
# rraFAN
predicted_lambdas_fan = forecast_lambda(data=sri_log_returnstest, dependent_var='FAN', conditional_variances= conditional_variances_fan_test, model_fit= rraFAN)
print(predicted_lambdas_fan)

In [ ]:
plt.figure(figsize=(10, 2))
plt.plot(predicted_lambdas_fan, label='fan')
plt.legend()

In [ ]:
print(predicted_lambdas_fan['predicted_lambda'].describe())

### TRA

In [ ]:
# tra_log_returnstest
# rraSPY
predicted_lambdas_spy = forecast_lambda(data=tra_log_returnstest, dependent_var='SPY', conditional_variances= conditional_variances_spy_test, model_fit= rraFAN)
print(predicted_lambdas_spy)

In [ ]:
plt.figure(figsize=(10, 2))
plt.plot(predicted_lambdas_spy, label='spy')
plt.legend()

In [ ]:
print(predicted_lambdas_spy['predicted_lambda'].describe())

# go garch (1,1) for all ETFS in 2 portfolio direclty in the out of sample period

In [ ]:
# Try to infer the frequency of the date index
inferred_freq = pd.infer_freq(sri_log_returnstest.index)
if inferred_freq is not None:
    sri_log_returnstest.index.freq = inferred_freq

inferred_freq = pd.infer_freq(tra_log_returnstest.index)
if inferred_freq is not None:
    tra_log_returnstest.index.freq = inferred_freq

In [ ]:
# handle NaN value
print("NaNs in each column:")
print(sri_log_returnstest.isna().sum())
# Check for infinite values
print("\nInfinite values in each column:")
print((np.isinf(sri_log_returnstest)).sum())
sri_log_returnstest.dropna(inplace=True)

In [ ]:
# handle NaN value
print("NaNs in each column:")
print(tra_log_returnstest.isna().sum())
# Check for infinite values
print("\nInfinite values in each column:")
print((np.isinf(tra_log_returnstest)).sum())
tra_log_returnstest.dropna(inplace=True)

### Fit AR(1) for each ETFs

In [ ]:
# SRI AR 1 for each ETF
# Fit AR(1) model to each SRI log returns
conditional_means = pd.DataFrame(index=sri_log_returnstest.index, columns=sri_log_returnstest.columns)
residuals = pd.DataFrame(index=sri_log_returnstest.index, columns=sri_log_returnstest.columns)
model_params = pd.DataFrame(index=['mean', 'theta', 'variance'], columns=sri_log_returnstest.columns)

for etf in sri_log_returns.columns:
    model = AutoReg(sri_log_returnstest[etf], lags=1, old_names=False)
    fitted_model = model.fit()
    conditional_means[etf] = fitted_model.fittedvalues
    residuals[etf] = fitted_model.resid

    # Store model parameters
    model_params.at['mean', etf] = fitted_model.params.get('const', 0)  # Safe access with default
    model_params.at['theta', etf] = fitted_model.params.get(etf + '.L1', 0)  # Safe access with default
    model_params.at['variance', etf] = fitted_model.sigma2

# Compute the sample covariance matrix Σ from the AR residuals
sri_covariance_matrix = residuals.cov()
sri_covariance_array = sri_covariance_matrix.values

# Output results
print(f"Model Parameters: (Mean, Theta, Variance):")
print(model_params)
print(f"Residual:")
print(residuals)
print("Sample Covariance Matrix:")
print(sri_covariance_matrix)

In [ ]:
# SRI AR 1 for each ETF
# Fit AR(1) model to each TRA log returns
# Initialization of storage for fitted values, residuals, and model parameters
conditional_means_tra = pd.DataFrame(index=tra_log_returnstest.index, columns=tra_log_returnstest.columns)
residuals_tra = pd.DataFrame(index=tra_log_returnstest.index, columns=tra_log_returnstest.columns)
model_params_tra = pd.DataFrame(index=['mean', 'theta', 'variance'], columns=tra_log_returnstest.columns)

# Fitting the AR(1) model
for etf in tra_log_returnstest.columns:
    model_tra = AutoReg(tra_log_returnstest[etf].dropna(), lags=1, old_names=False)
    fitted_model_tra = model_tra.fit()
    conditional_means_tra[etf] = fitted_model_tra.fittedvalues
    residuals_tra[etf] = fitted_model_tra.resid

    # Store model parameters
    model_params_tra.at['mean', etf] = fitted_model_tra.params.get('const', 0)
    model_params_tra.at['theta', etf] = fitted_model_tra.params.get('L1', 0)
    model_params_tra.at['variance', etf] = fitted_model_tra.sigma2

# Computing the sample covariance matrix from the AR residuals
tra_covariance_matrix = residuals_tra.cov()
tra_covariance_array = tra_covariance_matrix.values

# Output results
print("Model Parameters: (Mean, Theta, Variance):")
print(model_params_tra)
print("\nResiduals:")
print(residuals_tra)
print("\nSample Covariance Matrix:")
print(tra_covariance_matrix)

### whitening data and decomposition to get the mixing matrix

In [ ]:
# step 2
# SRI
# Eigenvalue decomposition of the provided covariance matrix 
eigenvalues, eigenvectors = eigh(sri_covariance_matrix)

# Whitening transformation
positive_eigenvalues = np.clip(eigenvalues, a_min=1e-10, a_max=None)
D_inv_sqrt = np.diag(1.0 / np.sqrt(positive_eigenvalues))
whitening_matrix = eigenvectors @ D_inv_sqrt @ eigenvectors.T
residuals_np = residuals.values.astype(float)
whitened_data_np = np.dot(residuals.values, whitening_matrix)
whitened_data = residuals.values @ whitening_matrix

#forward fill any remaining NaN values #backward fill any remaining NaN values
whitened_data_df = pd.DataFrame(whitened_data, index=residuals.index, columns=residuals.columns)
whitened_data_df.ffill(inplace=True)
whitened_data_df.bfill(inplace=True)

ica = FastICA(whiten=False)  # No need to whiten as data is already pre-whitened
try:
    independent_components = ica.fit_transform(whitened_data)
    mixing_matrix = ica.mixing_
    unmixing_matrix = np.linalg.inv(mixing_matrix)

    print("Independent Components:\n", independent_components)
    print("Mixing Matrix A:\n", mixing_matrix)
    print("Unmixing Matrix W (A^(-1)):\n", unmixing_matrix)
except ValueError as e:
    print("Error during FastICA:", e)

In [ ]:
# TRA
tra_covariance_matrix_np = tra_covariance_matrix.values.astype(float)
eigenvalues, eigenvectors = eigh(tra_covariance_matrix_np)

positive_eigenvalues = np.clip(eigenvalues, a_min=1e-10, a_max=None)
D_inv_sqrt = np.diag(1.0 / np.sqrt(positive_eigenvalues))
whitening_matrix = eigenvectors @ D_inv_sqrt @ eigenvectors.T

whitened_data = residuals_tra.values @ whitening_matrix
whitened_data_df = pd.DataFrame(whitened_data, index=residuals_tra.index, columns=residuals_tra.columns)
whitened_data_df.ffill(inplace=True)
whitened_data_df.bfill(inplace=True)

ica = FastICA(whiten=False)  # Since data is already pre-whitened
try:
    independent_components_tra = ica.fit_transform(whitened_data_df)
    mixing_matrix_tra = ica.mixing_
    unmixing_matrix_tra = np.linalg.inv(mixing_matrix_tra)

    print("Independent Components TRA:\n", independent_components_tra)
    print("Mixing Matrix A TRA:\n", mixing_matrix_tra)
    print("Unmixing Matrix W (A^(-1)) TRA:\n", unmixing_matrix_tra)
except ValueError as e:
    print("Error during FastICA:", e)

### finding the factorial GARCH (1,1)

In [ ]:

# step 3 sri GARCH (1,1) 
column_names = [f"Factor_{i+1}" for i in range(independent_components.shape[1])]
independent_components_df = pd.DataFrame(independent_components, columns=column_names)
H_f_t = []
# Dictionary to store models and results
garch_models = {}
results_summary = {}

# Loop through each column in the DataFrame which represents a factor.
for factor in independent_components_df.columns:
    # Initialize and fit a GARCH(1,1) model to the factor data.
    model = arch_model(independent_components_df[factor], mean='Zero', vol='Garch', p=1, q=1)
    res = model.fit(update_freq=0, disp='off')  # Fit the model quietly without updating the console.
    H_f_t.append(res.conditional_volatility**2)
    # Store the fitted model and results in dictionaries for easy access later.
    garch_models[factor] = res
    results_summary[factor] = {
        'omega': res.params.get('omega', float('nan')),  # Constant variance component
        'alpha': res.params.get('alpha[1]', float('nan')),  # Response of volatility to shocks
        'beta': res.params.get('beta[1]', float('nan')),  # Volatility persistence
        'Log Likelihood': res.loglikelihood  # Model's log likelihood
    }

# Optionally, convert the results summary to a DataFrame for better visualization and analysis.
results_df = pd.DataFrame(results_summary).T  # Transpose to have factors as rows and parameters as columns.
print(results_df)

In [ ]:
# step 3 tra GARCH !! 
column_names = [f"Factor_{i+1}" for i in range(independent_components_tra.shape[1])]
independent_components_df = pd.DataFrame(independent_components_tra, columns=column_names)

# Dictionary to store models and results
garch_models = {}
results_summary = {}
H_f_t_tra = []
# Loop through each column in the DataFrame which represents a factor.
for factor in independent_components_df.columns:
    # Initialize and fit a GARCH(1,1) model to the factor data.
    model = arch_model(independent_components_df[factor], mean='Zero', vol='Garch', p=1, q=1)
    res = model.fit(update_freq=0, disp='off')  # Fit the model quietly without updating the console.
    H_f_t_tra.append(res.conditional_volatility**2)
    # Store the fitted model and results in dictionaries for easy access later.
    garch_models[factor] = res
    results_summary[factor] = {
        'omega': res.params.get('omega', float('nan')),  # Constant variance component
        'alpha': res.params.get('alpha[1]', float('nan')),  # Response of volatility to shocks
        'beta': res.params.get('beta[1]', float('nan')),  # Volatility persistence
        'Log Likelihood': res.loglikelihood  # Model's log likelihood
    }

# Optionally, convert the results summary to a DataFrame for better visualization and analysis.
results_df_tra = pd.DataFrame(results_summary).T  # Transpose to have factors as rows and parameters as columns.
print(results_df_tra)

### compute the conditional covariance matrices Σ_t

In [ ]:
# Step 4: Compute the conditional covariance matrices Σ_t of SRI
H_f_t = np.array(H_f_t).T  # Transpose to have time on rows and factors on columns
conditional_covariances_sri = []
for t in range(H_f_t.shape[0]):
    # Construct the diagonal matrix of variances for time t
    H_t = np.diag(H_f_t[t])
    # Calculate the conditional covariance matrix Σ_t
    Σ_t = mixing_matrix@ H_t @ mixing_matrix.T
    conditional_covariances_sri.append(Σ_t)
conditional_covariances_sri = np.array(conditional_covariances_sri)  # Optional: convert list to array for easier handling

print("Example of Conditional Covariance Matrix Σ_t of SRI at time t=0:")
print(conditional_covariances_sri[0])
print(conditional_covariances_sri.shape)

In [ ]:
# Step 4: Compute the conditional covariance matrices Σ_t of TRA
H_f_t_tra = np.array(H_f_t_tra).T  # Transpose to have time on rows and factors on columns
conditional_covariances_tra = []
for t in range(H_f_t_tra.shape[0]):
    # Construct the diagonal matrix of variances for time t
    H_t_tra = np.diag(H_f_t_tra[t])
    # Calculate the conditional covariance matrix Σ_t
    Σ_t = mixing_matrix_tra@H_t_tra@ mixing_matrix_tra.T
    conditional_covariances_tra.append(Σ_t)
conditional_covariances_tra = np.array(conditional_covariances_tra)  # Optional: convert list to array for easier handling

print("Example of Conditional Covariance Matrix Σ_t of TRA:")
print(conditional_covariances_tra)

In [ ]:
print('shape of covariance matrix')
print(conditional_covariances_tra.shape)

### estimate the dynamic third order skewness & fourth order kurtosis

In [ ]:
# STEP 5 for SRI
f_t_sri = independent_components  # Should be an array of shape (time_points, n_components)
n_components_sri = f_t_sri.shape[1]  # Number of components/factors
# Initialize arrays to store co-moments
M3_t_sri = np.zeros((n_components_sri, n_components_sri, n_components_sri))
M4_t_sri = np.zeros((n_components_sri, n_components_sri, n_components_sri, n_components_sri))

# Assume A is your mixing matrix
A = mixing_matrix  # Ensure this is correctly defined and corresponds with f_t in shape

# Compute higher-order co-moments directly in the loops
for i in range(n_components_sri):
    for j in range(n_components_sri):
        for k in range(n_components_sri):
            # Compute the third-order co-moment for each combination of factors
            third_moment = np.mean(f_t_sri[:, i] * f_t_sri[:, j] * f_t_sri[:, k])
            M3_t_sri[i, j, k] = np.sum(A[i, :] * A[j, :] * A[k, :] * third_moment)

for i in range(n_components_sri):
    for j in range(n_components_sri):
        for k in range(n_components_sri):
            for l in range(n_components_sri):
                # Compute the fourth-order co-moment for each combination of factors
                fourth_moment = np.mean(f_t_sri[:, i] * f_t_sri[:, j] * f_t_sri[:, k] * f_t_sri[:, l])
                M4_t_sri[i, j, k, l] = np.sum(A[i, :] * A[j, :] * A[k, :] * A[l, :] * fourth_moment)
# Print results
print("Dynamic third-order co-moment array SRI, M3_t:")
print(M3_t_sri[1])  # Example of the third co-moment array for the second set of factors
print("Dynamic fourth-order co-moment array SRI, M4_t:")
print(M4_t_sri[1])  # Example of the fourth co-moment array for the second set of factors

In [ ]:
# STEP 5 for TRA
f_t = independent_components_tra  # Should be an array of shape (time_points, n_components)
n_components = f_t.shape[1]  # Number of components/factors
# Initialize arrays to store co-moments
M3_t_tra = np.zeros((n_components, n_components, n_components))
M4_t_tra = np.zeros((n_components, n_components, n_components, n_components))

# Assume A is your mixing matrix
A = mixing_matrix_tra  # Ensure this is correctly defined and corresponds with f_t in shape

# Compute higher-order co-moments directly in the loops
for i in range(n_components):
    for j in range(n_components):
        for k in range(n_components):
            # Compute the third-order co-moment for each combination of factors
            third_moment = np.mean(f_t[:, i] * f_t[:, j] * f_t[:, k])
            M3_t_tra[i, j, k] = np.sum(A[i, :] * A[j, :] * A[k, :] * third_moment)

for i in range(n_components):
    for j in range(n_components):
        for k in range(n_components):
            for l in range(n_components):
                # Compute the fourth-order co-moment for each combination of factors
                fourth_moment = np.mean(f_t[:, i] * f_t[:, j] * f_t[:, k] * f_t[:, l])
                M4_t_tra[i, j, k, l] = np.sum(A[i, :] * A[j, :] * A[k, :] * A[l, :] * fourth_moment)

# Print results
print("Dynamic third-order co-moment array TRA, M3_t:")
print(M3_t_tra[1])  # Example of the third co-moment array for the second set of factors
print("Dynamic fourth-order co-moment array TRA, M4_t:")
print(M4_t_tra[1])  # Example of the fourth co-moment array for the second set of factors

# portfolio optimization

### SRI

In [ ]:
predicted_lambdas_fan = pd.DataFrame(predicted_lambdas_fan,columns=['predicted_lambda'])
predicted_lambdas_fan.index = pd.date_range(start=predicted_lambdas_fan.index[0], periods=len(predicted_lambdas_fan), freq='B')
print(predicted_lambdas_fan)

In [ ]:
print(M3_t_sri.shape)

In [ ]:
print(M4_t_sri.shape)

In [ ]:
# Trimming conditional_covariances_tra to match the length of predicted_lambdas_spy
conditional_covariances_sri = conditional_covariances_sri[:500]
sri_ticker_1 = [ 'CXSE','FAN','PBW'] 
# Assuming M3_t_sri and M4_t_sri are static, we only need one set of these matrices
assert M3_t_sri.shape == (3, 3, 3), f"Expected skewness matrix shape (3, 3, 3) but got {M3_t_sri.shape}"
assert M4_t_sri.shape == (3, 3, 3, 3), f"Expected kurtosis matrix shape (3, 3, 3, 3) but got {M4_t_sri.shape}"

# Risk exposure function
def risk_exposure(weights, lambda_value, cov_matrix, skewness_matrix, kurtosis_matrix):
    weights_tf = tf.constant(weights, dtype=tf.float32)
    
    # Portfolio variance
    cov_matrix_tf = tf.constant(cov_matrix, dtype=tf.float32)
    portfolio_variance = tf.tensordot(weights_tf, tf.tensordot(cov_matrix_tf, weights_tf, axes=1), axes=1)
    
    # Portfolio skewness
    skewness_matrix_tf = tf.constant(skewness_matrix, dtype=tf.float32)
    portfolio_skewness = tf.einsum('i,j,k,ijk->', weights_tf, weights_tf, weights_tf, skewness_matrix_tf)
    
    # Portfolio kurtosis
    kurtosis_matrix_tf = tf.constant(kurtosis_matrix, dtype=tf.float32)
    portfolio_kurtosis = tf.einsum('i,j,k,l,ijkl->', weights_tf, weights_tf, weights_tf, weights_tf, kurtosis_matrix_tf)
    
    # Calculate risk exposure
    risk_exposure_value = (0.5 * lambda_value * portfolio_variance - 
                           (lambda_value**2 / 6) * portfolio_skewness + 
                           (lambda_value**3 / 24) * (portfolio_kurtosis - 3))
    
    # Ensure the return value is a scalar
    return risk_exposure_value.numpy().item()

# Constants and Data
n_assets = 3  # Number of assets in the portfolio
sri_ticker = ['PBW', 'CXSE', 'FAN']

# Ensure the lambda series has the correct length
assert len(predicted_lambdas_fan) == 500, f"Expected 500 but got {len(predicted_lambdas_fan)}"
assert conditional_covariances_sri.shape == (500, 3, 3), f"Expected (500, 3, 3) but got {conditional_covariances_sri.shape}"

# Bounds and constraints
bounds = [(0, 1)] * n_assets
cons = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}  # Sum of weights must be 1

# Initialize DataFrame to store optimal weights with ticker names as columns
optimal_weights_sri = pd.DataFrame(index=predicted_lambdas_fan.index, columns=sri_ticker_1)

# Dynamic Optimization Over Time
for idx, (date, lambda_value) in enumerate(predicted_lambdas_fan['predicted_lambda'].items()):
    cov_matrix = conditional_covariances_sri[idx]
    result = minimize(risk_exposure, [1/n_assets] * n_assets, 
                      args=(lambda_value, cov_matrix, M3_t_sri, M4_t_sri),
                      method='SLSQP', bounds=bounds, constraints=cons)
    
    # Debugging information
    print(f"Date: {date}, Success: {result.success}, Message: {result.message}, Weights: {result.x}")
    
    if result.success:
        optimal_weights_sri.loc[date] = result.x
    else:
        optimal_weights_sri.loc[date] = [np.nan] * n_assets  # Handle failed optimization

# Output results
print("Optimal Weights Over Time SRI:")
print(optimal_weights_sri)

# Descriptive statistics of the optimal weights
print("Descriptive Statistics of Optimal Weights SRI:")
print(optimal_weights_sri.describe())

### TRA

In [ ]:
predicted_lambdas_spy = pd.DataFrame(predicted_lambdas_spy,columns=['predicted_lambda'])
predicted_lambdas_spy.index = pd.date_range(start=predicted_lambdas_spy.index[0], periods=len(predicted_lambdas_spy), freq='B')
print(predicted_lambdas_spy)

In [ ]:
print(M3_t_tra.shape)

In [ ]:
print(M4_t_tra.shape)

In [ ]:
# Trimming conditional_covariances_tra to match the length of predicted_lambdas_spy
conditional_covariances_tra = conditional_covariances_tra[:500]
traditional_ticker = ['EWH','URTH', 'SPY']
# Assuming M3_t_sri and M4_t_sri are static, we only need one set of these matrices
assert M3_t_tra.shape == (3, 3, 3), f"Expected skewness matrix shape (3, 3, 3) but got {M3_t_tra.shape}"
assert M4_t_tra.shape == (3, 3, 3, 3), f"Expected kurtosis matrix shape (3, 3, 3, 3) but got {M4_t_tra.shape}"

# Risk exposure function
def risk_exposure(weights, lambda_value, cov_matrix, skewness_matrix, kurtosis_matrix):
    weights_tf = tf.constant(weights, dtype=tf.float32)
    
    # Portfolio variance
    cov_matrix_tf = tf.constant(cov_matrix, dtype=tf.float32)
    portfolio_variance = tf.tensordot(weights_tf, tf.tensordot(cov_matrix_tf, weights_tf, axes=1), axes=1)
    
    # Portfolio skewness
    skewness_matrix_tf = tf.constant(skewness_matrix, dtype=tf.float32)
    portfolio_skewness = tf.einsum('i,j,k,ijk->', weights_tf, weights_tf, weights_tf, skewness_matrix_tf)
    
    # Portfolio kurtosis
    kurtosis_matrix_tf = tf.constant(kurtosis_matrix, dtype=tf.float32)
    portfolio_kurtosis = tf.einsum('i,j,k,l,ijkl->', weights_tf, weights_tf, weights_tf, weights_tf, kurtosis_matrix_tf)
    
    # Calculate risk exposure
    risk_exposure_value = (0.5 * lambda_value * portfolio_variance - 
                           (lambda_value**2 / 6) * portfolio_skewness + 
                           (lambda_value**3 / 24) * (portfolio_kurtosis - 3))
    
    # Ensure the return value is a scalar
    return risk_exposure_value.numpy().item()

# Constants and Data
n_assets = 3  # Number of assets in the portfolio

# Ensure the lambda series has the correct length
assert len(predicted_lambdas_spy) == 500, f"Expected 500 but got {len(predicted_lambdas_spy)}"
assert conditional_covariances_tra.shape == (500, 3, 3), f"Expected (500, 3, 3) but got {conditional_covariances_tra.shape}"

# Bounds and constraints
bounds = [(0, 1)] * n_assets
cons = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}  # Sum of weights must be 1

# Initialize DataFrame to store optimal weights with ticker names as columns
optimal_weights_tra = pd.DataFrame(index=predicted_lambdas_fan.index, columns=traditional_ticker)

# Dynamic Optimization Over Time
for idx, (date, lambda_value) in enumerate(predicted_lambdas_spy['predicted_lambda'].items()):
    cov_matrix = conditional_covariances_tra[idx]
    result = minimize(risk_exposure, [1/n_assets] * n_assets, 
                      args=(lambda_value, cov_matrix, M3_t_tra, M4_t_tra),
                      method='SLSQP', bounds=bounds, constraints=cons)
    
    # Debugging information
    print(f"Date: {date}, Success: {result.success}, Message: {result.message}, Weights: {result.x}")
    
    if result.success:
        optimal_weights_tra.loc[date] = result.x
    else:
        optimal_weights_tra.loc[date] = [np.nan] * n_assets  # Handle failed optimization

# Output results
print("Optimal Weights Over Time TRA:")
print(optimal_weights_tra)

# Descriptive statistics of the optimal weights
print("Descriptive Statistics of Optimal Weights TRA:")
print(optimal_weights_tra.describe())

# Portfolio return

### SRI

In [ ]:
print(sri_log_returnstest.index)

In [ ]:
print(optimal_weights_sri)

In [ ]:
sri_log_returnstest.index = pd.to_datetime(sri_log_returnstest.index)

# Print the initial DataFrame and its index to confirm
print("Original DataFrame:")
print(sri_log_returnstest)
print(sri_log_returnstest.index)

# Trim the DataFrame starting from '2018-01-04'
trim_log_returns = sri_log_returnstest.loc['2018-01-04':]

# Print the trimmed DataFrame and its index to confirm
print("\nTrimmed DataFrame:")
print(trim_log_returns)
print(trim_log_returns.index)

In [ ]:
print("Columns of trim_log_returns:")
print(trim_log_returns.columns)

print("Columns of optimal_weights_sri:")
print(optimal_weights_sri.columns)

In [ ]:
# Ensure the indices match
optimal_weights_sri = optimal_weights_sri.reindex(trim_log_returns.index)

# Fill any potential missing values (if any dates are missing in the weights data)
optimal_weights_sri.ffill(inplace=True)

# Remove the column name from trim_log_returns to match optimal_weights_sri
trim_log_returns.columns.name = None

# Rename and reorder the columns of optimal_weights_sri to match those of trim_log_returns
optimal_weights_sri = optimal_weights_sri[trim_log_returns.columns]

# Verify the column names after renaming and reordering
print("Columns of optimal_weights_sri after renaming and reordering:")
print(optimal_weights_sri.columns)
print("Columns of trim_log_returns:")
print(trim_log_returns.columns)

In [ ]:
# Ensure the indices match
optimal_weights_sri = optimal_weights_sri.reindex(trim_log_returns.index)

# Fill any potential missing values (if any dates are missing in the weights data)
optimal_weights_sri.ffill(inplace=True)

def calculate_portfolio_returns_from_weights(log_returns, optimal_weights):
    # Check alignment of indices
    if not log_returns.index.equals(optimal_weights.index):
        raise ValueError("Indices of log returns and optimal weights must match.")
    
    # Convert weights DataFrame from object to float if necessary
    optimal_weights = optimal_weights.astype(float)
    
    # Ensure optimal weights and log returns have the same columns
    if list(optimal_weights.columns) != list(log_returns.columns):
        raise ValueError("Columns of optimal weights must match columns of log returns.")
    
    # Compute the portfolio returns by element-wise multiplication and then sum along the columns
    portfolio_returns_ = (log_returns * optimal_weights).sum(axis=1)
    
    return portfolio_returns_

# Calculate the portfolio returns
try:
    sri_portfolio = calculate_portfolio_returns_from_weights(trim_log_returns, optimal_weights_sri)
    print(sri_portfolio)
except ValueError as e:
    print(f"Error: {e}")

In [ ]:
def portfolio_summarize(portfolio_returns):
    # Ensure portfolio_returns is a one-dimensional Series
    if isinstance(portfolio_returns, pd.DataFrame):
        portfolio_returns = portfolio_returns.stack()
    
    # Calculate mean and variance for the entire series
    mean_return = portfolio_returns.mean()
    variance_return = portfolio_returns.var()
    
    # Calculate kurtosis and skewness using scipy.stats functions
    kurt = scipy_kurtosis(portfolio_returns, fisher=False)
    skewness = scipy_skew(portfolio_returns)
    
    # Calculate Jarque-Bera test
    jb_stat, jb_p_value = jarque_bera(portfolio_returns)
    
    # Print the results
    print(f"Mean of Portfolio Returns: {mean_return}")
    print(f"Variance of Portfolio Returns: {variance_return}")
    print(f"Kurtosis of Portfolio Returns: {kurt}")
    print(f"Skewness of Portfolio Returns: {skewness}")
    print(f"Jarque-Bera Test Statistic: {jb_stat}")
    print(f"Jarque-Bera Test p-value: {jb_p_value}")

# Summarize the portfolio returns
portfolio_summarize(sri_portfolio)

In [ ]:
plt.figure(figsize=(6, 4))  
plt.hist(sri_portfolio, label='SRI')
plt.legend()
plt.show()

### TRA

In [ ]:
print(tra_log_returnstest.head())

In [ ]:
print(optimal_weights_tra)

In [ ]:
tra_log_returnstest.index = pd.to_datetime(tra_log_returnstest.index)

# Print the initial DataFrame and its index to confirm
print("Original DataFrame:")
print(tra_log_returnstest)
print(tra_log_returnstest.index)

# Trim the DataFrame starting from '2018-01-04'
trim_log_returns_tra = tra_log_returnstest.loc['2018-01-04':]

# Print the trimmed DataFrame and its index to confirm
print("\nTrimmed DataFrame:")
print(trim_log_returns_tra)
print(trim_log_returns_tra.index)

In [ ]:
print("Columns of trim_log_returns tra:")
print(trim_log_returns_tra.columns)

print("Columns of optimal_weights_tra:")
print(optimal_weights_tra.columns)

In [ ]:
# Ensure the indices match
optimal_weights_tra = optimal_weights_tra.reindex(trim_log_returns_tra.index)

# Fill any potential missing values (if any dates are missing in the weights data)
optimal_weights_tra.ffill(inplace=True)

# Remove the column name from trim_log_returns to match optimal_weights_sri
trim_log_returns_tra.columns.name = None

# Rename and reorder the columns of optimal_weights_sri to match those of trim_log_returns
optimal_weights_tra = optimal_weights_tra[trim_log_returns_tra.columns]

# Verify the column names after renaming and reordering
print("Columns of optimal_weights_tra after renaming and reordering:")
print(optimal_weights_tra.columns)
print("Columns of trim_log_returns tra:")
print(optimal_weights_tra.columns)

In [ ]:
# Ensure the indices match
# Fill any potential missing values (if any dates are missing in the weights data)
optimal_weights_tra.ffill(inplace=True)

In [ ]:
tra_portfolio= calculate_portfolio_returns_from_weights(trim_log_returns_tra, optimal_weights_tra)
print(tra_portfolio)

In [ ]:
portfolio_summarize(tra_portfolio)

In [ ]:
plt.figure(figsize=(6, 4))  
plt.hist(tra_portfolio, label='TRA', color='green')
plt.legend()
plt.show()